In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-03 20:16:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.36MB/s    in 0.2s    

2023-02-03 20:16:43 (4.36 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date" ])
review_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [17]:
# Create the customers_table DataFrame
# customers_df = df.groupby("customer_id").agg({"count"}).withColumnRenamed("customer_id", "customer_count")
# customers_df.show(5)


customers_df = df.select(['customer_id'])
customers_df = df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
customers_df.show(5)













+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
+-----------+--------------+
only showing top 5 rows



In [19]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows



In [28]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [29]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [35]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challengeaws.cvutbg8zkhld.us-east-1.rds.amazonaws.com:5432/challengeaws"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [39]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [40]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [41]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [42]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)